In [1]:
import pandas as pd
import unittest

In [7]:
testData = {'Area': (['Indonesia'] * 3 + ['China'] * 3 + ['UK'] * 3) * 3,
            'Item': ['Apples'] * 9 + ['Grapes'] * 9 + ['Beehives'] * 9,
            'Element': ['Yield', 'Production', 'Area'] * 6 + ['Yield', 'Stocks', 'Count'] * 3}

testDf = pd.DataFrame(testData)
testDf.head()

,Area,Item,Element
0,Indonesia,Apples,Yield
1,Indonesia,Apples,Production
2,Indonesia,Apples,Area
3,China,Apples,Yield
4,China,Apples,Production
